In [188]:
import requests
import os
import pytest
import requests
import json
import datetime
from datetime import datetime, timedelta
from email.utils import formatdate
import certifi
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

### Bank norwegian API

In [100]:
bn_url = "https://api-sandbox.banknorwegian.com/openbanking"

client_secret = YOUR_CLIENT_SECRET

# Define the date 30 days in the future in the correct format
formatted_valid_until_date = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')

# Define the current date for the 'date' header, following the RFC 7231 specification
formatted_current_date = formatdate(timeval=None, localtime=False, usegmt=True)

# Client ID
client_id = YOUR_CLIENT_ID

In [170]:
# Create consent payload
consent_payload = {
    "validUntil": formatted_valid_until_date,
    "recurringIndicator": True,
    "access": {
        "accounts": [],
        "balances": [],
        "transactions": []
    },
    "frequencyPerDay": 5,
    "combinedServiceIndicator": True
}

# Create headers
headers = {
    'certificate-validation': 'false',
    'content-type': 'application/json',
    'Date': formatted_current_date,  # Use the current date in the proper format
    'PSU-Device-ID': 'Iphone',
    'PSU-ID': '1527',
    'PSU-IP-Address': '127.0.0.1',
    'PSU-IP-Port': '3301',
    'PSU-User-Agent': 'Insomnia',
    'Region-ID': 'NO',
    'Request-ID': 'd1707de7-b5d2-44c7-bfc4-f85a8324fd60',
    'signature-validation': 'false',
    'TPP-Client-ID': client_id,
    'TPP-Redirect-URI': 'http://localhost:3000/',
    'TPP-Client-Secret': client_secret
}

consent_id = None
try:
    response = requests.post(
        "https://api-sandbox.banknorwegian.com/openbanking/consents",
        headers=headers,
        json=consent_payload,
        verify=False
    )
    
    # Check if the request was successful
    if response.status_code == 201:  # HTTP 201 indicates that the request has been fulfilled and has resulted in a new resource being created
        response_data = response.json()
        print("Response Data:", json.dumps(response_data, indent=4))
        
        # Extract and save the consent_id
        consent_id = response_data.get("consentId", None)
        if consent_id:
            print(f"Consent ID: {consent_id}")
    
    else:
        print(f"Received unexpected status code {response.status_code}: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Users/Vetle/miniconda3/envs/ban438/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-sandbox.banknorwegian.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Response Data: {
    "consentStatus": "Received",
    "consentId": "e70311e0-3994-4a29-8de9-2fe14bbd3b70",
    "_links": {
        "scaRedirect": {
            "href": "https://identity-no-sandbox.bn.basefarm.net/connect/authorize?response_type=code&response_mode=query&client_id=47760def26a04e6187b4dec0fe87144d&scope=offline_access readaccounts initiatepayment&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2F&consent_id=e70311e0-3994-4a29-8de9-2fe14bbd3b70&prompt=login&code_challenge=<YOUR_CODE_CHALLENGE>&code_challenge_method=S256"
        }
    }
}
Consent ID: e70311e0-3994-4a29-8de9-2fe14bbd3b70


In [172]:
code_verifier = base64.urlsafe_b64encode(os.urandom(32)).decode('utf-8')
code_verifier = code_verifier[:43]

# Generating a code_challenge based on the code_verifier
m = hashlib.sha256()
m.update(code_verifier.encode('utf-8'))
code_challenge = base64.urlsafe_b64encode(m.digest()).decode('utf-8').replace('=', '')

# Generate the authorization URL with the code_challenge
auth_url = "https://identity-no-sandbox.bn.basefarm.net/connect/authorize?"
params = {
    'response_type': 'code',
    'response_mode': 'query',
    'client_id': '47760def26a04e6187b4dec0fe87144d',
    'scope': 'offline_access readaccounts initiatepayment',
    'redirect_uri': 'http://localhost:3000/',
    'consent_id': consent_id,
    'prompt': 'login',
    'code_challenge': code_challenge,
    'code_challenge_method': 'S256'
}
auth_url += urllib.parse.urlencode(params)

print("Authorization URL:", auth_url)

Authorization URL: https://identity-no-sandbox.bn.basefarm.net/connect/authorize?response_type=code&response_mode=query&client_id=47760def26a04e6187b4dec0fe87144d&scope=offline_access+readaccounts+initiatepayment&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2F&consent_id=e70311e0-3994-4a29-8de9-2fe14bbd3b70&prompt=login&code_challenge=loWtDo1AkYErpZK4sDwmUlDboGpc4XhaMDZMsQ1ifCw&code_challenge_method=S256


In [173]:
code = "37C9A7798780BAFEC5417D96F5C016D8262A8FB7B460AB7D4BF11488813B242E-1"

In [174]:
# Define the API endpoint
url = "https://identity-no-sandbox.bn.basefarm.net/connect/token"

# Define the headers
headers = {
    'content-type': 'application/x-www-form-urlencoded'
}

# Define the data payload
data = {
    'client_id': client_id,
    'client_secret': client_secret,
    'code': code,
    'grant_type': 'authorization_code',
    'redirect_uri': 'http://localhost:3000/',
    'code_verifier': code_verifier
}

# Make the request
try:
    response = requests.post(url, headers=headers, data=data)

    # Check for success
    if response.status_code == 200:
        print("Successfully received token!")
        print(response.json())
    else:
        print(f"Failed to get token. Status Code: {response.status_code}")
        print(f"Response Headers: {response.headers}")
        print(f"Response Content: {response.content.decode('utf-8')}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Successfully received token!
{'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjhDOEMxRTM5MDQwMkRBMUNGQkYwRDk0OEZENDZCQjZBIiwidHlwIjoiYXQrand0In0.eyJpc3MiOiJodHRwczovL2lkZW50aXR5LW5vLXNhbmRib3guYm4uYmFzZWZhcm0ubmV0IiwibmJmIjoxNjk2NTE0MzE1LCJpYXQiOjE2OTY1MTQzMTUsImV4cCI6MTY5NjUxNzkxNSwiYXVkIjpbImFjY291bnRzYXBpIiwicGF5bWVudHNhcGkiXSwic2NvcGUiOlsicmVhZGFjY291bnRzIiwiaW5pdGlhdGVwYXltZW50Iiwib2ZmbGluZV9hY2Nlc3MiXSwiYW1yIjpbImV4dGVybmFsIl0sImNsaWVudF9pZCI6IjQ3NzYwZGVmMjZhMDRlNjE4N2I0ZGVjMGZlODcxNDRkIiwic3ViIjoiMTUyNyIsImF1dGhfdGltZSI6MTY5NjUxNDI5MywiaWRwIjoiMTAiLCJjb25zZW50X2lkIjoiZTcwMzExZTAtMzk5NC00YTI5LThkZTktMmZlMTRiYmQzYjcwIiwic2lkIjoiQTAwNTQ1NDVGMjc4OTAwNzdDQURDMzFEMjE1QjUxODIiLCJqdGkiOiJDNkIwMTE1REZFQTc3NUJDNURBRjU2OUQyRTlFMzQ2NiJ9.hjOUzYsC67ZCPycalfYZGIElRLvpWtM9wdR4zuG7qmo7o3RlWqBk7r68gWQ1M3sGX3Q162-_eUFBpuj-_PTgBJxAad2rLw7i7rQT-JG_NOs4csV5GMrVEWOPlSJDMHtB5BHMTx-1ITiardEFb4J6KXqhzkrGwhS5XD213SWdU9KLg0HYUgPqgqJVL30t6rS3bB1O4GyHSaWYwcKC4glLa087llEfYgWGx6g6TpYtJ1SY0U9Bj3ZMMEJo4POJiAyIFkd5

In [175]:
# getting accounts

# Saving access_token and other details
token_data = response.json()
access_token = token_data['access_token']
expires_in = token_data['expires_in']
refresh_token = token_data['refresh_token']
scope = token_data['scope']

In [178]:
# Headers for the GET request to fetch accounts
account_headers = {
    'Authorization': f'Bearer {access_token}',
    'accept': 'application/json',
    'consent-id': consent_id,
    'date': datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT'),
    'psu-device-id': 'Iphone',
    'psu-id': '1527',
    'psu-ip-address': '127.0.0.1',
    'psu-ip-port': '3301',
    'psu-user-agent': 'Insomnia',
    'region-id': 'NO',
    'request-id': 'd1707de7-b5d2-44c7-bfc4-f85a8324fd60',
    'certificate-validation': 'false',
    'signature-validation': 'false'
}

# URL for the GET request to fetch accounts
account_url = 'https://api-sandbox.banknorwegian.com/openbanking/accounts?withBalance=true'

try:
    account_response = requests.get(account_url, headers=account_headers, verify=False)
    
    # If successful, save the resourceId as account_id
    if account_response.status_code == 200:
        accounts_data = account_response.json()
        account_id = accounts_data['accounts'][0]['resourceId']
        print(f"Account ID fetched: {account_id}")
        
        resource_ids = [account['resourceId'] for account in accounts_data['accounts']]

        # Pretty print the accounts data
        print("Accounts Data:")
        print(json.dumps(accounts_data, indent=4))

    else:
        print(f"Failed to fetch accounts. Status Code: {account_response.status_code}")
        print("Response Content:", account_response.content.decode('utf-8'))

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Users/Vetle/miniconda3/envs/ban438/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-sandbox.banknorwegian.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Account ID fetched: 9a91c94f-1ab4-4ba5-9ec2-f7d8eb3bc575
Accounts Data:
{
    "accounts": [
        {
            "resourceId": "9a91c94f-1ab4-4ba5-9ec2-f7d8eb3bc575",
            "bban": "93557027325",
            "currency": "NOK",
            "name": "My savings 1",
            "ownerName": "Alias Undercover Sandbox",
            "product": "SavingsAccount",
            "cashAccountType": "CurrentAccount",
            "status": "Enabled",
            "remittanceInformation": {
                "resourceId": "9a91c94f-1ab4-4ba5-9ec2-f7d8eb3bc575",
                "accountType": "BBAN",
                "bban": "93557027325"
            },
            "balances": [
                {
                    "balanceAmount": {
                        "currency": "NOK",
                        "amount": "31415.92"
                    },
                    "balanceType": "ClosingBooked",
                    "creditLimitIncluded": false,
                    "referenceDate": "2023-10-05T00:00:00

In [192]:
formatted_current_date = datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')

df_transactions = pd.DataFrame(columns=['Transaction Date', 'Amount'])

# Define headers
headers = {
    'Authorization': f'Bearer {access_token}',  # Include the access_token here
    'accept': 'application/json',
    'consent-id': consent_id,
    'date': formatted_current_date,
    'psu-device-id': 'Iphone',
    'psu-id': '1527',
    'psu-ip-address': '127.0.0.1',
    'psu-ip-port': '3301',
    'psu-user-agent': 'Insomnia',
    'region-id': 'NO',
    'request-id': 'd1707de7-b5d2-44c7-bfc4-f85a8324fd60',
    'certificate-validation': 'false',
    'signature-validation': 'false'
}


# Loop through each resource_id to fetch transactions
for resource_id in resource_ids:
    url = f'https://api-sandbox.banknorwegian.com/openbanking/accounts/{resource_id}/transactions?bookingStatus=both'

    try:
        response = requests.get(url, headers=headers, verify=False)
        
        if response.status_code == 200:
            json_data = response.json()
            transactions_list = json_data.get('transactions', {}).get('booked', [])

            for transaction in transactions_list:
                transaction_date = transaction['transactionDate']
                amount = transaction['transactionAmount']['amount']
                
                # Append to DataFrame
                df_transactions = df_transactions.append({'Transaction Date': transaction_date, 'Amount': amount}, ignore_index=True)
        
        else:
            print(f"Failed to fetch transactions for Account {resource_id}. Status Code: {response.status_code}")
            print("Response Content:", response.content.decode('utf-8'))

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching transactions for Account {resource_id}: {e}")

# Show DataFrame
print(df_transactions)

{'accountReference': {'bban': '93557027325', 'currency': 'NOK'}, 'transactions': {'booked': [], 'pending': []}, '_links': {'account': {'href': '/accounts/9a91c94f-1ab4-4ba5-9ec2-f7d8eb3bc575'}}}
{'accountReference': {'bban': '93557027333', 'currency': 'NOK'}, 'transactions': {'booked': [], 'pending': []}, '_links': {'account': {'href': '/accounts/ef16cb7f-bed6-4b43-ba11-6669bfe61c35'}}}
{'accountReference': {'bban': '93557027341', 'currency': 'NOK'}, 'transactions': {'booked': [], 'pending': []}, '_links': {'account': {'href': '/accounts/ac23d46d-fb6b-4f6d-9e58-1d84b21a308f'}}}
{'accountReference': {'bban': '00566198004', 'currency': 'NOK'}, 'transactions': {'booked': [{'transactionId': '1145304419', 'entryReference': '1145304419', 'transactionDate': '2023-10-04T00:00:00.0000000', 'bookingDate': '2023-10-06T00:00:00.0000000', 'valueDate': '2023-10-21T00:00:00.0000000', 'transactionAmount': {'currency': 'NOK', 'amount': '-504'}, 'exchangeRate': [{'currencyFrom': 'NOK', 'rate': '1.00', '

{'accountReference': {'bban': '93557027368', 'currency': 'NOK'}, 'transactions': {'booked': [], 'pending': []}, '_links': {'account': {'href': '/accounts/be783f9f-e59a-414a-8e82-243b1716adee'}}}
                Transaction Date    Amount
0    2023-10-04T00:00:00.0000000      -504
1    2023-10-04T00:00:00.0000000     -1994
2    2023-10-04T00:00:00.0000000    -129.8
3    2023-10-03T00:00:00.0000000       -21
4    2023-10-03T00:00:00.0000000       -29
..                           ...       ...
111  2023-07-15T00:00:00.0000000   -1860.9
112  2023-07-14T00:00:00.0000000      -679
113  2023-07-14T00:00:00.0000000    -153.9
114  2023-07-14T00:00:00.0000000  -1398.06
115  2023-07-14T00:00:00.0000000       -35

[116 rows x 2 columns]


In [194]:
pd.set_option('display.max_rows', None)

df_transactions

,Transaction Date,Amount
0,2023-10-04T00:00:00.0000000,-504
1,2023-10-04T00:00:00.0000000,-1994
2,2023-10-04T00:00:00.0000000,-129.8
3,2023-10-03T00:00:00.0000000,-21
4,2023-10-03T00:00:00.0000000,-29
5,2023-10-02T00:00:00.0000000,-213.9
6,2023-10-02T00:00:00.0000000,-21
7,2023-10-01T00:00:00.0000000,500
8,2023-09-30T00:00:00.0000000,-584.02
9,2023-09-29T00:00:00.0000000,-164.6


In [187]:
df_transactions['Amount'] = df_transactions['Amount'].astype(float)

total_amount = df_transactions['Amount'].sum()

total_amount

-17040.069999999996